In [ ]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
from textblob import TextBlob
import re

%store -r

In [ ]:
# Import the restored data
# %run PrepareData.ipynb

In [ ]:
# Reset the index of the dataframe
tsla_tweets_df = tsla_tweets_df.reset_index()

In [ ]:
# Clean the text
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)#Remove @mentions
    text = re.sub(r'#', '', text)# Remove '#' hash tag
    text = re.sub(r'RT[\s]+', '', text)# Remove RT
    text = re.sub(r'http?:\/\/\S+', '', text)# Remove hyperlink
    text = re.sub(r'\$[A-Za-z0-9]+', '', text)# Remove $
    text = re.sub(r'https?:\/\/\S+', '', text)# Remove hyperlink
    
    return text

tsla_tweets_df['body'] = tsla_tweets_df['body'].apply(cleanTxt)

In [ ]:
# Creaet a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
# Apply the models to the cleaned data
tsla_tweets_df['Subjectivity'] = tsla_tweets_df['body'].apply(getSubjectivity)
tsla_tweets_df['Polarity'] = tsla_tweets_df['body'].apply(getPolarity)

In [ ]:
# Check the sentiment analysis result
tsla_tweets_df[['Subjectivity','Polarity']]